# Koalas

## 0. Setting Up Koalas/Apache Spark

In [ ]:
!pip install koalas
!pip install pyspark
!pip install databricks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.2/593.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=cbad591d736a755f9d2c36a6d87db028ab74fc6952a2a6f39669f6d47c0b6b57
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
pip install git+https://github.com/databricks/koalas.git

  Cloning https://github.com/databricks/koalas.git to /tmp/pip-req-build-v96rc5qb
  Running command git clone --filter=blob:none --quiet https://github.com/databricks/koalas.git /tmp/pip-req-build-v96rc5qb
  Resolved https://github.com/databricks/koalas.git to commit 476518d0537f5b9fe844276aed3c916f7e945019
  Preparing metadata (setup.py) ... done
  Created wheel for koalas: filename=koalas-1.8.2-py3-none-any.whl size=390753 sha256=a291813c533c7b9eef18e95770a0258e1e71f85dd0ac5190013568df47173299
  Stored in directory: /tmp/pip-ephem-wheel-cache-ctlgu8j2/wheels/19/b8/18/aaf319849edd6c0929e548ef1c634d4f5dbd76780a10352818
Successfully built koalas
  Attempting uninstall: koalas
    Found existing installation: koalas 0.32.0
    Uninstalling koalas-0.32.0:
      Successfully uninstalled koalas-0.32.0


In [ ]:
import databricks.koalas as ks
import numpy as np

## 1. Data selection and loading

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"muhammadhazimas","key":"f565ee0e8bcb4dd61c17357a8b9a681f"}'}

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

rm: cannot remove '/root/.kaggle': No such file or directory


In [ ]:
!kaggle datasets download -d cisautomotiveapi/large-car-dataset

100% 823M/825M [00:06<00:00, 237MB/s]
100% 825M/825M [00:06<00:00, 124MB/s]


In [ ]:
!unzip /content/large-car-dataset.zip

Archive:  /content/large-car-dataset.zip
  inflating: CIS_Automotive_Kaggle_Sample.csv  


In [ ]:
path ='/content/CIS_Automotive_Kaggle_Sample.csv'

In [ ]:
%time
df_k = ks.read_csv(path, nrows=1000000)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs


In [ ]:
df_k['stockNum'] = df_k['stockNum'].replace('N/A', None)

In [ ]:
df_k

## 2. Data preparation and cleaning

### 2.1 data type optimization

In [ ]:
df_k.dtypes

vin                  object
stockNum             object
firstSeen            object
lastSeen             object
msrp                  int32
                      ...  
vf_WheelBaseType     object
vf_WheelSizeFront     int32
vf_WheelSizeRear      int32
vf_Wheels             int32
vf_Windows            int32
Length: 156, dtype: object

In [ ]:
%time
for column in df_k.columns:
    if df_k[column].dtype == 'float64':
        df_k[column] = df_k[column].astype('float')
    elif df_k[column].dtype == 'int64':
        df_k[column] = df_k[column].astype('int')

df_k['firstSeen'] = ks.to_datetime(df_k['firstSeen'])
df_k['lastSeen'] = ks.to_datetime(df_k['lastSeen'])

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs


/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/functions.py:407: UserWarning:

In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.



In [ ]:
df_k.dtypes

vin                      object
stockNum                 object
firstSeen            datetime64
lastSeen             datetime64
msrp                      int32
                        ...    
vf_WheelBaseType         object
vf_WheelSizeFront         int32
vf_WheelSizeRear          int32
vf_Wheels                 int32
vf_Windows                int32
Length: 156, dtype: object

### 2.2 handle missing value

In [ ]:
for column in df_k.columns:
  if df_k[column].isnull().sum() <= 1000:
    print("Column", column, "has", df_k[column].isnull().sum(), "missing values")
print()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

### 2.2.1 dropping rows with significant NaN

In [ ]:
df_k.shape

(1000000, 156)

In [ ]:
%time
df_k = df_k.dropna(subset=['stockNum'])

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs


In [ ]:
df_k.shape

(646534, 156)

#### 2.2.2 missing value replacement

In [ ]:
df_k['stockNum'].isnull().sum()

353466

In [ ]:
%time
df_k['color'].fillna('N/A',inplace=True)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs


In [ ]:
df_k.isnull().sum()

/usr/local/lib/python3.10/dist-packages/databricks/koalas/internal.py:1430: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, col in reset_index.iteritems():
/usr/local/lib/python3.10/dist-packages/databricks/koalas/internal.py:1356: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [


vin                                            0
stockNum                                       0
firstSeen                                      0
lastSeen                                       0
msrp                                           0
askPrice                                       0
mileage                                        0
isNew                                          0
color                                          0
interiorColor                                  0
brandName                                    109
modelName                                    709
dealerID                                       0
vf_ABS                                    581738
vf_ActiveSafetySysNote                    856757
vf_AdaptiveCruiseControl                  823106
vf_AdaptiveDrivingBeam                    843025
vf_AdaptiveHeadlights                     998136
vf_AdditionalErrorText                    994571
vf_AirBagLocCurtain                       369701
vf_AirBagLocFront   

### 2.3 identifying duplicate rows

In [ ]:
%time
df_k.duplicated().sum()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


0

### Drop duplicate Column

In [ ]:
df_k.drop(['vf_Model'],axis = 1 )

## 3. Exploratory Data Analysis and Visualization

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
%matplotlib inline

### 3.1 statistic of numeric *column*

In [ ]:
%time
df_k.describe()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


,msrp,askPrice,mileage,dealerID,vf_Axles,vf_BasePrice,vf_BatteryA,vf_BatteryCells,vf_BatteryKWh,vf_BatteryKWh_to,vf_BatteryPacks,vf_BatteryV,vf_BedLengthIN,vf_ChargerPowerKW,vf_CurbWeightLB,vf_DisplacementCC,vf_DisplacementCI,vf_DisplacementL,vf_Doors,vf_EngineCycles,vf_EngineCylinders,vf_EngineHP,vf_EngineHP_to,vf_EngineKW,vf_MakeID,vf_ManufacturerId,vf_ModelID,vf_ModelYear,vf_SAEAutomationLevel,vf_SeatRows,vf_Seats,vf_TopSpeedMPH,vf_TrackWidth,vf_TrailerLength,vf_TransmissionSpeeds,vf_WheelBaseLong,vf_WheelBaseShort,vf_WheelSizeFront,vf_WheelSizeRear,vf_Wheels,vf_Windows
count,9.986390e+05,9.986390e+05,9.986390e+05,998639.000000,375300.000000,326068.000000,0.0,0.0,206.000000,18.000000,10917.000000,1553.000000,922.000000,192.000000,22430.000000,990353.000000,990353.000000,990353.000000,877497.000000,86540.000000,924437.000000,602438.000000,32448.000000,602275.000000,9.985300e+05,998530.000000,997930.000000,998523.000000,31.0,349749.000000,371162.000000,171998.000000,11576.000000,9.000000,218158.000000,15730.000000,365575.000000,304648.000000,304586.000000,388693.000000,49521.000000
mean,3.582409e+04,2.308832e+04,2.131962e+04,19286.480678,2.000011,33943.684242,NaN,NaN,22.881553,75.555556,1.100027,231.478429,65.608460,30.625000,3998.023897,3251.826037,198.563871,3.254641,4.040110,4.002011,5.201457,232.299057,291.402432,173.213565,1.479665e+15,1155.923518,3598.102723,2015.544231,1.0,2.154454,5.337311,127.082867,63.795871,30.333333,6.922666,151.873857,113.448329,17.674746,17.678567,4.000077,4.101169
std,4.000968e+06,2.083077e+04,4.441363e+04,11977.645083,0.010062,12884.214921,NaN,NaN,15.159572,20.714366,0.300050,82.852144,5.644139,5.339275,579.444144,7238.482213,441.830600,7.240300,0.575375,0.064824,1.433568,81.889410,489.331246,61.075326,8.536558e+17,1387.105848,4276.023131,4.286941,0.0,0.414673,1.004202,14.395930,3.233794,14.857658,1.430297,18.505348,124.122974,1.240238,1.231992,0.114254,0.438300
min,0.000000e+00,0.000000e+00,0.000000e+00,378.000000,1.000000,13.900000,NaN,NaN,8.800000,40.000000,1.000000,4.000000,60.000000,10.000000,2335.000000,0.000000,0.000000,0.000000,2.000000,4.000000,1.000000,18.200000,87.000000,13.571740,4.400000e+02,955.000000,1685.000000,1982.000000,1.0,1.000000,2.000000,84.000000,58.300000,16.000000,1.000000,102.200000,11.290000,2.000000,14.000000,1.000000,4.000000
25%,1.145800e+04,1.096200e+04,0.000000e+00,8315.000000,2.000000,24800.000000,NaN,NaN,12.000000,60.000000,1.000000,240.000000,60.000000,32.000000,3520.000000,2000.000000,122.047488,2.000000,4.000000,4.000000,4.000000,170.000000,148.000000,126.769000,4.640000e+02,984.000000,1850.000000,2014.000000,1.0,2.000000,5.000000,118.000000,61.800000,29.000000,6.000000,149.400000,106.200000,17.000000,17.000000,4.000000,4.000000
50%,2.208100e+04,2.067100e+04,0.000000e+00,20676.000000,2.000000,30995.000000,NaN,NaN,24.000000,85.000000,1.000000,240.000000,66.000000,32.000000,4054.000000,2500.000000,152.559360,2.500000,4.000000,4.000000,4.000000,220.000000,284.000000,164.054000,4.740000e+02,994.000000,1945.000000,2017.000000,1.0,2.000000,5.000000,130.000000,64.000000,29.000000,6.000000,156.800000,111.000000,18.000000,18.000000,4.000000,4.000000
75%,3.448000e+04,3.247900e+04,2.577000e+04,31925.000000,2.000000,40395.000000,NaN,NaN,24.000000,90.000000,1.000000,270.000000,67.000000,32.000000,4330.000000,3600.000000,219.685479,3.600000,4.000000,4.000000,6.000000,288.000000,314.000000,214.761600,4.820000e+02,1034.000000,3134.000000,2019.000000,1.0,2.000000,5.000000,130.000000,64.400000,29.000000,8.000000,156.900000,114.800000,18.000000,18.000000,4.000000,4.000000
max,1.878820e+09,5.877140e+06,9.999999e+06,39974.000000,8.000000,189900.000000,NaN,NaN,81.000000,90.000000,2.000000,360.000000,79.000000,32.000000,5815.000000,200000.000000,12204.748819,200.000000,6.000000,8.000000,12.000000,840.000000,6500.000000,626.388000,4.924965e+20,19306.000000,27729.000000,2021.000000,1.0,5.000000,12.000000,205.000000,79.200000,67.000000,10.000000,176.000000,30500.000000,22.000000,22

### 3.2 Visualization

#### 3.2.1 Bar graph

In [ ]:
%time
df_k['sales_data'] = df_k['lastSeen'].dt.year.astype(str) +'-'+df_k['lastSeen'].dt.month.astype(str)

Sale_month = df_k.groupby(['sales_data']).agg({'vin':'count'}).reset_index()

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.11 µs


In [ ]:
Sale_month

,sales_data,vin
0,2019-10,25935
1,2018-10,18837
2,2019-4,31984
3,2019-11,26851
4,2020-3,19408
5,2018-7,32343
6,2018-9,20342
7,2020-5,27683
8,2019-9,27921
9,2019-6,32407


In [ ]:
%time
fig = px.bar(Sale_month, x='sales_data', y='vin', labels={'vin': 'Number of Sales'}, title='Monthly Sales Count')
fig.show()

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.11 µs


#### 3.2.2 Scatterplot

In [ ]:
%time
df_k['priceDiff'] =  df_k['msrp'] - df_k['askPrice']
x = df_k[df_k['priceDiff']]

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs


In [ ]:
%time
x.plot.scatter(x='msrp', y='askPrice')

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


#### 3.2.3 Line graph

In [ ]:
%time
df_k['Combined'] = ks.to_datetime(df_k['sales_month'].astype(str) + '/' + df_k['sales_year'].astype(str), format='%m/%Y')
graph_data = df_k.groupby(['Combined'])['askPrice'].mean().sort_index()

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.11 µs


/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/functions.py:407: UserWarning:

In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.



In [ ]:
%time
graph_data.plot.line()

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.34 µs


## 4. Answering Question

### QUESTION 1 : List the top 5 car that is most bought by customer

In [ ]:
%%time
brands = df_k.groupby(['brandName']).agg({'vin':'count'})
brands = brands.sort_values(by='vin', ascending=False)
data = brands.head(5)
data

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


,vin
brandName,
FORD,89498
CHEVROLET,68499
HONDA,59601
TOYOTA,59597
NISSAN,45652


In [ ]:
%%time
fig = data.plot.bar()
fig.update_layout(title_text='Top 5 Brands With The Most Cars Sold',xaxis_title='Brand Name', yaxis_title='Number of Cars Sold')
fig.show()

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.63 µs


### QUESTION 2 : On average, in days, how fast does it take for a car to be sold,list the top 10 dealer with the fastest sell

In [ ]:
%%time
df_k['avg_num_of_day_to_sell'] = df_k['lastSeen']-df_k['firstSeen']

pre_data = df_k.groupby(['dealerID']).agg({'avg_num_of_day_to_sell':'mean'})
data = pre_data['avg_num_of_day_to_sell'].nsmallest(10)
data = data.head(10)
data = data / (3600 * 24)
data

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs


/usr/local/lib/python3.10/dist-packages/databricks/koalas/base.py:368: UserWarning:

Note that there is a behavior difference of timestamp subtraction. The timestamp subtraction returns an integer in seconds, whereas pandas returns 'timedelta64[ns]'.



dealerID
11542    20.800000
38721    23.750000
24145    32.889095
30637    33.914783
15945    35.323576
37673    36.189655
39934    37.893343
35489    38.177720
30655    39.813636
8989     42.943015
Name: avg_num_of_day_to_sell, dtype: float64

### QUESTION 3 : How many new cars is sold based on year

In [ ]:
%%time
df_k['sales_year'] = df_k['lastSeen'].dt.year

newsold = df_k.query('isNew == True').groupby(['sales_year']).agg({'isNew':'count'})
newsold = newsold.sort_index(ascending=True)
newsold

CPU times: user 523 ms, sys: 85.4 ms, total: 608 ms
Wall time: 3.5 s


,isNew
sales_year,
2018,72892
2019,122060
2020,34495


In [ ]:
%%time
fig = newsold.plot.pie(y='isNew')
fig.update_layout(title_text='Number of New Cars Sold Based On Year')

CPU times: user 526 ms, sys: 56.6 ms, total: 582 ms
Wall time: 1min 15s


### QUESTION 4 : List the top 3 dealer with the most sales over the year

In [ ]:
%%time
df_k['dealerID'] = df_k['dealerID'].astype(str)
top_dealer = df_k.groupby(['dealerID']).agg({'askPrice':'sum'})
dealer_data = top_dealer.sort_values(by='askPrice', ascending=False)
dealer_data = dealer_data.head(10)
dealer_data

CPU times: user 397 ms, sys: 54.5 ms, total: 451 ms
Wall time: 2.18 s


,askPrice
dealerID,
12477,1590205703
31925,1587534013
22349,438730782
22329,399171661
37755,309209175
7081,286318080
4656,279607500
35585,279125011
35444,265527335


In [ ]:
%%time
dealer_data.plot(kind='bar')

CPU times: user 576 ms, sys: 35.3 ms, total: 611 ms
Wall time: 1min 14s
